In [1]:
import pandas as pd
train=pd.read_csv('/home/xiaoguzai/程序/剧本角色情感识别/数据/train_dataset_v2.tsv', sep='\t')

In [2]:
print(train['emotions'])

0        0,0,0,0,0,0
1        0,0,0,0,0,0
2        0,0,0,0,0,0
3        0,0,0,0,0,0
4        0,0,0,0,0,0
            ...     
42785    0,0,0,0,0,0
42786    0,3,0,0,0,0
42787    2,3,0,0,0,0
42788    2,3,0,0,0,0
42789    0,0,0,0,0,0
Name: emotions, Length: 42790, dtype: object


In [3]:
content_dict = {}
for index in range(len(train['content'])):
    currentindex = train['id'][index]
    currentindex = currentindex.split('_')
    str1 = currentindex[0]
    while len(str1) < 5:
        str1 = '0'+str1
    str4 = currentindex[3]
    while len(str4) < 4:
        str4 = '0'+str4
    resultindex = str1+'_'+currentindex[1]+'_'+currentindex[2]+'_'+str4
    content_dict[resultindex] = train['content'][index]

In [4]:
def sortedDictValues(adict): 
    keys = adict.keys() 
    keys = sorted(keys)
    print('keys = ')
    print(keys[0:10])
    new_content_dict = {}
    for data in keys:
        new_content_dict[data] = content_dict[data]
    return new_content_dict 
new_content_dict = sortedDictValues(content_dict)

keys = 
['01171_0001_A_0001', '01171_0001_A_0002', '01171_0001_A_0003', '01171_0001_A_0004', '01171_0001_A_0005', '01171_0001_A_0006', '01171_0001_A_0007', '01171_0001_A_0008', '01171_0001_A_0009', '01171_0001_A_0010']


In [5]:
inv_new_content_dict = {}
for data in new_content_dict:
    inv_new_content_dict[new_content_dict[data]] = data

In [6]:
keys = new_content_dict.keys()
keys = list(keys)
dict_keys_index = {}
for index in range(len(keys)):
    #dict_keys_index[index] = keys[index]
    dict_keys_index[keys[index]] = index

In [7]:
content,emotions = [],[]
dicts = {}
inv_dicts = {}
label_num = 0
characters = []
label1,label2,label3,label4,label5,label6 = [],[],[],[],[],[]
for index in range(len(train['content'])):
    if pd.isna(train['emotions'][index]) == False:
        content.append(train['content'][index])
        emotions.append(train['emotions'][index])
        characters.append(train['character'][index])
        current_emotion = train['emotions'][index].split(',')
        label1.append(int(current_emotion[0]))
        label2.append(int(current_emotion[1]))
        label3.append(int(current_emotion[2]))
        label4.append(int(current_emotion[3]))
        label5.append(int(current_emotion[4]))
        label6.append(int(current_emotion[5]))

In [8]:
set1 = set(label1)
dict1,dict2,dict3,dict4,dict5,dict6 = {},{},{},{},{},{}
for item in set1:
    dict1.update({item:label1.count(item)})
set2 = set(label2)
for item in set2:
    dict2.update({item:label2.count(item)})
set3 = set(label3)
for item in set3:
    dict3.update({item:label3.count(item)})
set4 = set(label4)
for item in set4:
    dict4.update({item:label4.count(item)})
set5 = set(label5)
for item in set5:
    dict5.update({item:label5.count(item)})
set6 = set(label6)
for item in set6:
    dict6.update({item:label6.count(item)})

In [9]:
dict1

{0: 35500, 1: 527, 2: 346, 3: 409}

In [10]:
dict2

{0: 34014, 1: 2057, 2: 479, 3: 232}

In [11]:
dict3

{0: 34568, 1: 1335, 2: 593, 3: 286}

In [12]:
dict4

{0: 33009, 1: 1998, 2: 1197, 3: 578}

In [13]:
dict5

{0: 33821, 1: 1541, 2: 1012, 3: 408}

In [14]:
dict6

{0: 31004, 1: 2821, 2: 1983, 3: 974}

In [15]:
import pandas as pd
import itertools
import json
import math
from bertmodels import Config
from nezha import Bert
vocab_file = r'/home/xiaoguzai/模型/bert/nezha-base/vocab.txt'
vocab_size = len(open(vocab_file,'r').readlines()) 
with open('/home/xiaoguzai/模型/bert/nezha-base/config.json','r',encoding='utf8')as fp:
    json_data = json.load(fp)
json_data['vocab_size'] = vocab_size
config = Config(**json_data)

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset,DataLoader
from tokenization import FullTokenizer
import numpy as np
tokenizer = FullTokenizer(vocab_file=vocab_file)
from tqdm import tqdm
config.with_mlm = False
#config.with_pooler = True
bertmodel = Bert(config)
import os
import random

---__init__ Nezha


In [16]:
def seed_torch(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
seed_torch(9)

In [17]:
from loader_nezha import load_bert_data
bertmodel = load_bert_data(bertmodel,'/home/xiaoguzai/模型/bert/nezha-base/pytorch_model.bin')

!!!load Pytorch model!!!
Done loading 196 NEZHA weights from: /home/xiaoguzai/模型/bert/nezha-base/pytorch_model.bin. Count of weights not found in the checkpoint was: [0]. Count of weights with mismatched shape: [0]
Unused weights from checkpoint: 
	cls.predictions.transform.dense.bias
	cls.seq_relationship.weight
	cls.predictions.transform.dense.weight
	bert.pooler.dense.weight
	bert.pooler.dense.bias
	cls.seq_relationship.bias
	cls.predictions.transform.LayerNorm.bias
	cls.predictions.bias
	cls.predictions.transform.LayerNorm.weight
	cls.predictions.decoder.weight


from loader_pretrain_weights import load_bert_data
bertmodel = load_bert_data(bertmodel,'/home/xiaoguzai/数据集/剧本角色识别/labeled_data+model_epoch=60.pth')

In [18]:
class ClassificationDataset(Dataset):
    def __init__(self,text,characters,label1,label2,label3,label4,label5,label6,maxlen,flag):
        self.text = text
        self.maxlen = maxlen
        self.characters = characters
        #self.label = [label1,label2,label3,label4,label5,label6]
        token_data,token_id,segment_id,mask_id = [],[],[],[]
        #sequence填充可以最后统一实现
        for index in tqdm(range(len(self.text))):
            current_text = text[index]
            current_character = characters[index]
            current_token = tokenizer.tokenize(current_text)
            current_str_index = inv_new_content_dict[current_text]

            current_key_index = dict_keys_index[current_str_index]
            while current_key_index != -1 and new_content_dict[keys[current_key_index]] == current_text:
                current_key_index = current_key_index-1
            #找寻前面语句的过程之中去重
            if current_key_index == -1:
                pre_content = ''
            else:
                pre_content = new_content_dict[keys[current_key_index]]
            #找到前面去重之后的语句内容
            new_str_index = keys[current_key_index]
            current_str_index = current_str_index.split('_')
            new_str_index = new_str_index.split('_')
            if current_str_index[0] != new_str_index[0]:
                pre_content = ''
            if current_str_index[1] != new_str_index[1]:
                pre_content = ''
            #不属于同一个剧本杀的剧本
            
            
            if str(current_character) == 'nan':
                current_character = '无'
            if pre_content == '':
                pre_content = '无'
            current_character_token = tokenizer.tokenize(current_character)
            pre_token = tokenizer.tokenize(pre_content)
            current_token = ["[CLS]"]+pre_token+["[SEP]"]+current_token+["[SEP]"]+current_character_token+["[SEP]"]

            current_id = tokenizer.convert_tokens_to_ids(current_token)            
            current_id = self.sequence_padding(current_id)
            token_data.append(current_token)
            token_id.append(current_id)
        self.token_data = token_data
        self.token_id = token_id
        self.label = [label1,label2,label3,label4,label5,label6]
        self.tensors = [torch.tensor(self.token_id,dtype=torch.long),
                 torch.tensor(self.label[0],dtype=torch.long),
                 torch.tensor(self.label[1],dtype=torch.long),
                 torch.tensor(self.label[2],dtype=torch.long),
                 torch.tensor(self.label[3],dtype=torch.long),
                 torch.tensor(self.label[4],dtype=torch.long),
                 torch.tensor(self.label[5],dtype=torch.long)]
        
    def __len__(self):
        return len(self.token_id)
    
    def __getitem__(self,index):
        return tuple(tensor[index] for tensor in self.tensors)

    def sequence_padding(self,inputs,padding = 0):
        length = self.maxlen
        if len(inputs) > length:
            inputs = inputs[:length]
        outputs = []
        pad_width = (0,length-len(inputs))
        x = np.pad(inputs,pad_width,'constant',constant_values=padding)
        return x

In [19]:
text = content
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
import numpy as np
ss = StratifiedKFold(n_splits=5,shuffle=True,random_state=2)
#建立4折交叉验证方法 查一下KFold函数的参数
text = np.array(text)
label1 = np.array(label1)
label2 = np.array(label2)
label3 = np.array(label3)
label4 = np.array(label4)
label5 = np.array(label5)
label6 = np.array(label6)
characters = np.array(characters)
for train_index,test_index in ss.split(text,label1):
    train_text = text[np.array(train_index)]
    test_text = text[test_index]
    train_characters = characters[train_index]
    test_characters = characters[test_index]
    train_label1 = label1[train_index]
    test_label1 = label1[test_index]
    train_label2 = label2[train_index]
    test_label2 = label2[test_index]
    train_label3 = label3[train_index]
    test_label3 = label3[test_index]
    train_label4 = label4[train_index]
    test_label4 = label4[test_index]
    train_label5 = label5[train_index]
    test_label5 = label5[test_index]
    train_label6 = label6[train_index]
    test_label6 = label6[test_index]
train_dataset = ClassificationDataset(train_text,train_characters,train_label1,train_label2,train_label3,train_label4,train_label5,train_label6,maxlen=200,flag=True)
test_dataset = ClassificationDataset(test_text,test_characters,test_label1,test_label2,test_label3,test_label4,test_label5,test_label6,maxlen=200,flag=False)
#到里面的classificationdataset才进行字符的切割以及划分
train_loader = DataLoader(train_dataset,batch_size=10,shuffle=True)
test_loader = DataLoader(test_dataset,batch_size=10)

100%|█████████████████████████████████████| 7356/7356 [00:01<00:00, 4872.38it/s]


In [20]:
class ClassificationModel(nn.Module):
    def __init__(self,model,config,n_labels):
        super(ClassificationModel,self).__init__()
        #self.embedding = nn.Embedding(30522,768)
        self.model = model
        self.fc1 = nn.Linear(config.embedding_size,128)
        self.activation = F.relu
        self.dropout = nn.Dropout(0.2)
        #self.activation = F.tanh
        self.fc2 = nn.Linear(128,n_labels)
        
    def forward(self,input_ids,segment_ids,input_mask):
        #outputs = self.embedding(input_ids)
        output = self.model(input_ids)
        #[64,128,768]
        output = self.dropout(output)
        output = output[:,0]
        output = self.fc1(output)
        output = self.activation(output)
        output = self.dropout(output)
        output = self.fc2(output)
        return output
    #之前这里少量return outputs返回值为None

In [21]:
def compute_multilabel_loss(x,model,label):
    logit = model(x,None,None)
    mseloss = 0
    loss_fn = torch.nn.MSELoss(reduce=True, size_average=True)
    logit = torch.transpose(logit, 0, 1)
    mseloss = loss_fn(logit,label)
    return mseloss

def compute_multilabel_loss(x,model,label):
    logit = model(x,None,None)
    mseloss = 0
    loss_fn = torch.nn.MSELoss(reduce=True, size_average=True)
    logit = torch.transpose(logit, 0, 1)
    mseloss = 0
    totaltimes = logit.shape[1]
    for index in range(len(logit)):
        for index1 in range(len(logit[index])):
            mseloss = mseloss+(logit[index][index1]-label[index][index1])**2
            if index == 5 and (label[index][index1] == 2 or label[index][index1] == 3):
                mseloss = mseloss+(logit[index][index1]-label[index][index1])**2
        #currentloss = loss_fn(logit,label)
        #if index == 5:
            #mseloss = mseloss+0.6*currentloss
    #mseloss = loss_fn(logit,label)
    return mseloss/totaltimes

In [22]:
from loader_bert import load_bert_data
from tqdm import tqdm
from colorama import Fore
n_label = 6
model = ClassificationModel(bertmodel,config,n_label)

In [23]:
import torch.nn.functional as F
from torch.optim.lr_scheduler import LambdaLR
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#model = model.to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=9e-6)
#不动态调整学习率的时候，lr=2e-5的时候最好值0.420293
def lr_lambda(epoch):
    if epoch > 5:
        return 1
    else:
        return 2/(epoch+1)
scheduler = LambdaLR(optimizer, lr_lambda)
print("初始化的学习率：", optimizer.defaults['lr'])
bestpoint = 0.0
for epoch in range(5):
    print('epoch {}'.format(epoch))
    train_loss = 0
    train_acc = 0
    
    model.train()
    
    model = model.to(device)
    model = nn.DataParallel(model)
    loss_fn = torch.nn.MSELoss(reduce=True,size_average=True)
    
    for batch_token_ids,batch_label1,batch_label2,batch_label3,batch_label4,batch_label5,batch_label6 in tqdm(train_loader):
        torch.set_printoptions(edgeitems=768)# 设置输出矩阵维度为768
        #print('batch_token_ids')
        #print(batch_token_ids)
        batch_token_ids = batch_token_ids.to(device)
        batch_labels = [batch_label1.numpy(),batch_label2.numpy(),batch_label3.numpy(),\
                        batch_label4.numpy(),batch_label5.numpy(),batch_label6.numpy()]
        batch_labels = torch.tensor(batch_labels,dtype=torch.float)
        batch_labels = batch_labels.to(device)
        #for index in range(len(batch_labels)):
        #    batch_labels[index] = batch_labels[index].to(device)
        optimizer.zero_grad()
        loss = compute_multilabel_loss(batch_token_ids,model,batch_labels)
        train_loss = train_loss+loss
        loss.backward()
        optimizer.step()
    scheduler.step()
    print("第%d个epoch的学习率：%f" % (epoch, optimizer.param_groups[0]['lr']))
    #注意必须从epoch=1开始，否则第0个没有学习率
    print('Train Loss: {:.6f}'.format(train_loss))
    
    model = model.to(device)
    model.eval()
    
    eval_true_label = [[],[],[],[],[],[]]
    eval_predict_label = [[],[],[],[],[],[]]
    
    eval_label_loss = [[0,0,0,0],\
                       [0,0,0,0],\
                       [0,0,0,0],\
                       [0,0,0,0]]
    #for batch_token_ids,batch_labels in tqdm(test_loader,bar_format='{l_bar}%s{bar}%s{r_bar}' % (Fore.BLUE, Fore.RESET)):
    for batch_token_ids,batch_label1,batch_label2,batch_label3,batch_label4,batch_label5,batch_label6 in tqdm(test_loader):
        batch_token_ids = batch_token_ids.to(device)
        batch_labels = [batch_label1,batch_label2,batch_label3,batch_label4,batch_label5,batch_label6]
        with torch.no_grad():
            output = model(batch_token_ids,None,None)
        for index in range(len(output)):
            #总的数据
            for index1 in range(len(output[index])):
                #对应的类别概率0~6
                abs0 = abs(output[index][index1]-0)
                abs1 = abs(output[index][index1]-1)
                abs2 = abs(output[index][index1]-2)
                abs3 = abs(output[index][index1]-3)
                currentdata = [abs0,abs1,abs2,abs3]
                current_label = currentdata.index(min(currentdata))
                #eval_predict_label[index1].append(current_label)
                current_predict = output[index][index1].item()
                if current_predict < 0.00:
                    current_predict = 0
                elif current_predict > 3:
                    current_predict = 3
                #当前类别的分类结果,这里append(output[index][index1])
                #直接放入对应概率值时效果最好,这里直接放入0,1,2,3对应的数值
                eval_predict_label[index1].append(current_predict)
        for index in range(len(batch_labels)):
            current_label = np.array(batch_labels[index].cpu()).tolist()
            eval_true_label[index].extend(current_label)
    criterion = nn.MSELoss()
    totalloss = 0

    for index in range(len(eval_true_label)):
        inputs = torch.autograd.Variable(torch.from_numpy(np.array(eval_predict_label[index])))
        target = torch.autograd.Variable(torch.from_numpy(np.array(eval_true_label[index])))
        for index1 in range(len(inputs)):
            abs0 = abs(inputs[index1]-0)
            abs1 = abs(inputs[index1]-1)
            abs2 = abs(inputs[index1]-2)
            abs3 = abs(inputs[index1]-3)
            currentdata = [abs0,abs1,abs2,abs3]
            current_label = currentdata.index(min(currentdata))
            true_label = target[index1].item()
            if current_label != true_label:
                eval_label_loss[true_label][current_label] = eval_label_loss[true_label][current_label]+1
                #对的预测为错误的
        current_loss = criterion(inputs.float(),target.float())
        current_loss = current_loss.item()
        print('index = %d,current_loss = %f'%(index,current_loss))
        totalloss = totalloss+current_loss
    
    #totalloss = totalloss/len(eval_predict_label)
    print('totalloss = ')
    print(totalloss)
    totalloss = totalloss/6
    totalloss = math.sqrt(totalloss)
    currentpoint = 1/(1+totalloss)
    #currentpoint = 1/(1+current_loss)
    print('current_point = ')
    print(currentpoint)
    if currentpoint > bestpoint:
        bestpoint = currentpoint
        torch.save(model,'best_score'+str(bestpoint)+'.pth')
    print('eval_label_loss = ')
    print(eval_label_loss)

初始化的学习率： 9e-06
epoch 0


/home/xiaoguzai/.local/lib/python3.9/site-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))
100%|███████████████████████████████████████| 2943/2943 [09:22<00:00,  5.23it/s]


第0个epoch的学习率：0.000009
Train Loss: 619.497192


100%|█████████████████████████████████████████| 736/736 [00:48<00:00, 15.33it/s]


index = 0,current_loss = 0.139167
index = 1,current_loss = 0.122403
index = 2,current_loss = 0.126275
index = 3,current_loss = 0.197817
index = 4,current_loss = 0.188617
index = 5,current_loss = 0.333467
totalloss = 
1.1077464371919632
current_point = 
0.6994575036879792
eval_label_loss = 
[[0, 1403, 31, 0], [1479, 0, 35, 0], [590, 397, 0, 0], [251, 240, 79, 0]]
epoch 1


...

第1个epoch的学习率：0.000006
Train Loss: 477.035645


100%|█████████████████████████████████████████| 736/736 [01:10<00:00, 10.40it/s]


index = 0,current_loss = 0.127342
index = 1,current_loss = 0.111529
index = 2,current_loss = 0.118145
index = 3,current_loss = 0.176136
index = 4,current_loss = 0.172255
index = 5,current_loss = 0.306858
totalloss = 
1.0122655406594276
current_point = 
0.7088456515462194
eval_label_loss = 
[[0, 1076, 72, 0], [1475, 0, 80, 0], [545, 359, 0, 0], [215, 192, 163, 0]]
epoch 2


 36%|██████████████                         | 1061/2943 [06:24<11:22,  2.76it/s]


KeyboardInterrupt: 

In [24]:
model = torch.load('best_score'+str(bestpoint)+'.pth')
test=pd.read_csv('/home/xiaoguzai/程序/剧本角色情感识别/数据/test_dataset.tsv', sep='\t')

In [25]:
content_dict = {}
for index in range(len(test['content'])):
    currentindex = test['id'][index]
    currentindex = currentindex.split('_')
    str1 = currentindex[0]
    while len(str1) < 5:
        str1 = '0'+str1
    str4 = currentindex[3]
    while len(str4) < 4:
        str4 = '0'+str4
    resultindex = str1+'_'+currentindex[1]+'_'+currentindex[2]+'_'+str4
    content_dict[resultindex] = test['content'][index]

In [26]:
def sortedDictValues(adict): 
    keys = adict.keys() 
    keys = sorted(keys)
    print('keys = ')
    print(keys[0:10])
    new_content_dict = {}
    for data in keys:
        new_content_dict[data] = content_dict[data]
    return new_content_dict 
new_content_dict = sortedDictValues(content_dict)

keys = 
['01597_0001_A_0001', '01597_0001_A_0002', '01597_0001_A_0003', '01597_0001_A_0004', '01597_0001_A_0005', '01597_0001_A_0006', '01597_0001_A_0007', '01597_0001_A_0008', '01597_0001_A_0009', '01597_0001_A_0010']


In [27]:
inv_new_content_dict = {}
for data in new_content_dict:
    inv_new_content_dict[new_content_dict[data]] = data

In [28]:
keys = new_content_dict.keys()
keys = list(keys)
dict_keys_index = {}
for index in range(len(keys)):
    #dict_keys_index[index] = keys[index]
    dict_keys_index[keys[index]] = index

In [29]:
content,emotions = [],[]
dicts = {}
inv_dicts = {}
label_num = 0
characters = []
for index in range(len(test['content'])):
        content.append(test['content'][index])
        characters.append(test['character'][index])

In [30]:
testtext = test['content']
testid = test['id']
testcharacter = test['character']
class TestDataset(Dataset):
    def __init__(self,text,character,maxlen):
        self.text = text
        self.maxlen = maxlen
        token_data,token_id,segment_id,mask_id = [],[],[],[]
        #sequence填充可以最后统一实现
        for index in tqdm(range(len(self.text))):
            current_text = text[index]
            current_character = characters[index]
            current_token = tokenizer.tokenize(current_text)
            current_str_index = inv_new_content_dict[current_text]

            current_key_index = dict_keys_index[current_str_index]
            while current_key_index != -1 and new_content_dict[keys[current_key_index]] == current_text:
                current_key_index = current_key_index-1
            #找寻前面语句的过程之中去重
            if current_key_index == -1:
                pre_content = ''
            else:
                pre_content = new_content_dict[keys[current_key_index]]
            
            new_str_index = keys[current_key_index]
            current_str_index = current_str_index.split('_')
            new_str_index = new_str_index.split('_')
            if current_str_index[0] != new_str_index[0]:
                pre_content = ''
            if current_str_index[1] != new_str_index[1]:
                pre_content = ''
            #不属于同一个剧本杀的剧本
            
            if str(current_character) == 'nan':
                current_character = '无'
            if pre_content == '':
                pre_content = '无'
            current_character_token = tokenizer.tokenize(current_character)
            pre_token = tokenizer.tokenize(pre_content)
            current_token = ["[CLS]"]+pre_token+["[SEP]"]+current_token+["[SEP]"]+current_character_token+["[SEP]"]
            
            current_id = tokenizer.convert_tokens_to_ids(current_token)            
            current_id = self.sequence_padding(current_id)
            token_data.append(current_token)
            token_id.append(current_id)
        self.token_data = token_data
        self.token_id = token_id
        #self.segment_id = sequence_padding(self.segment_id,maxlen)
        #self.mask_id = sequence_padding(self.mask_id,maxlen)
        self.tensors = [torch.tensor(self.token_id,dtype=torch.long)]
        
    def __len__(self):
        return len(self.token_id)
    
    def __getitem__(self,index):
        return tuple(tensor[index] for tensor in self.tensors)
    
    def sequence_padding(self,inputs,padding = 0):
        length = self.maxlen
        if len(inputs) > length:
            inputs = inputs[:length]
        outputs = []
        pad_width = (0,length-len(inputs))
        x = np.pad(inputs,pad_width,'constant',constant_values=padding)
        return x

In [31]:
test_dataset = TestDataset(testtext,testcharacter,maxlen=200)
test_loader = DataLoader(test_dataset,batch_size=10,shuffle=False)

100%|███████████████████████████████████| 21376/21376 [00:04<00:00, 5110.17it/s]


## 修改测试集的评分标准

In [32]:
model = model.to(device)
model.eval()
eval_loss = 0.
eval_acc = 0.
eval_predict_label = []
index = []
pred = [[],[],[],[],[],[]]
current_index = 0
for batch_token_ids in tqdm(test_loader):
    batch_token_ids = batch_token_ids[0].to(device)
    with torch.no_grad():
        output = model(batch_token_ids,None,None)
        for index in range(len(output)):
            #总的数据
            for index1 in range(len(output[index])):
                #对应的类别概率0~6
                abs0 = abs(output[index][index1]-0)
                abs1 = abs(output[index][index1]-1)
                abs2 = abs(output[index][index1]-2)
                abs3 = abs(output[index][index1]-3)
                currentdata = [abs0,abs1,abs2,abs3]
                current_label = currentdata.index(min(currentdata))
                #eval_predict_label[index1].append(current_label)
                current_predict = output[index][index1].item()
                
                if current_predict < 0.00:
                    current_predict = 0.0
                elif current_predict > 3:
                    current_predict = 3.0
                
                pred[index1].append(current_predict)
                #eval_predict_label[index1].append(current_predict)
                #当前类别的分类结果,这里append(output[index][index1])
                #直接放入对应概率值时效果最好,这里直接放入0,1,2,3对应的数值
for index in range(len(pred[0])):
    eval_predict_label.append(str(pred[0][index])+','+str(pred[1][index])+','+str(pred[2][index])+','+str(pred[3][index])+','+str(pred[4][index])+','+str(pred[5][index]))
result_data = []
for index in range(len(testid)):
    result_data.append([testid[index],eval_predict_label[index]])
#pd.DataFrame({"id":testid,"label":eval_predict_label}).to_csv("/home/xiaoguzai/代码/剧本角色情感识别/数据集/crossentropy"+str(bestpoint)+"result.csv",index=False)

100%|███████████████████████████████████████| 2138/2138 [04:20<00:00,  8.19it/s]


a=np.array([[1,2],[3,4]])
b=np.array([[2,3],[4,5]])
loss_fn = torch.nn.MSELoss(reduce=False, size_average=False)
input = torch.autograd.Variable(torch.from_numpy(a))
target = torch.autograd.Variable(torch.from_numpy(b))
loss = loss_fn(input.float(),target.float())

In [33]:
import csv
with open(r'/home/xiaoguzai/程序/剧本角色情感识别/数据/nezha_base'+str(bestpoint)+"result.csv", 'w') as f:
    tsv_w = csv.writer(f, delimiter='\t')
    tsv_w.writerow(['id', 'emotion'])  # 单行写入
    tsv_w.writerows(result_data)  # 多行写入